# Decision Tree
순도 , 엔트로피 , 정보획득량(분할 전후 엔트로피 차이)

# XGBoost
- 여러 DT 조합 (앙상블 개념)

# 앙상블 기법 분류 2가지
- 배깅 (Bagging) -> 여러 결과들의 빈도수를 조사해 최종 출력을 결정
- 부스팅 (Boosting)

In [1]:
import pandas as pd
import numpy as np

In [2]:
filepath = 'D:/downloads/titanic/'

In [3]:
train = pd.read_csv(filepath+'train.csv')
train.shape

(891, 12)

In [4]:
test = pd.read_csv(filepath+'test.csv')
test.shape

(418, 11)

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [14]:
train['Age_mean'] = train.Age.where(train.Age.notnull(),train.Age.mean())
train.head(7)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_mean
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,22.000000
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,38.000000
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,26.000000
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,35.000000
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,35.000000
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,29.699118
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,54.000000


In [16]:
test['Age_mean'] = test.Age.where(test.Age.notnull(),test.Age.mean())
test['Age_mean'].isnull().sum()

0

In [63]:
train['Embarked_S'] = train['Embarked']=='S'
train['Embarked_C'] = train['Embarked']=='C'
train['Embarked_Q'] = train['Embarked']=='Q'
test['Embarked_S'] = test['Embarked']=='S'
test['Embarked_C'] = test['Embarked']=='C'
test['Embarked_Q'] = test['Embarked']=='Q'

In [23]:
train['FamilySize'] = train['SibSp']+train['Parch']+1
train['FamilySize'].value_counts()

1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
Name: FamilySize, dtype: int64

In [22]:
def sml(fm_size):
    if fm_size==1:
        return 'S'
    return 'L' if fm_size>=5 else 'M'

train['Family']=train['FamilySize'].apply(sml)
train['Family_S']=train['Family']=='S'
train['Family_M']=train['Family']=='M'
train['Family_L']=train['Family']=='L'
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked,Age_mean,Embarked_S,Embarked_C,Embarked_Q,FamilySize,Family,Family_S,Family_M,Family_L
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,S,22.0,True,False,False,2,M,False,True,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,C,38.0,False,True,False,2,M,False,True,False


In [24]:
test['FamilySize'] = test['SibSp']+test['Parch']+1
test['Family']=test['FamilySize'].apply(sml)
test['Family_S']=test['Family']=='S'
test['Family_M']=test['Family']=='M'
test['Family_L']=test['Family']=='L'
test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_mean,Embarked_S,Embarked_C,Embarked_Q,FamilySize,Family,Family_S,Family_M,Family_L
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,34.5,True,False,False,1,S,True,False,False
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,47.0,False,True,False,2,M,False,True,False


In [25]:
train.Family.value_counts()

S    537
M    292
L     62
Name: Family, dtype: int64

In [26]:
test.Family.value_counts()

S    253
M    145
L     20
Name: Family, dtype: int64

In [28]:
train['Pclass']=train['Pclass'].astype('category')
train['Pclass']

0      3
1      1
2      3
3      1
4      3
      ..
886    2
887    1
888    3
889    1
890    3
Name: Pclass, Length: 891, dtype: category
Categories (3, int64): [1, 2, 3]

In [52]:
train['Gender'] = np.where(train.Sex=='male',True,False)
test['Gender'] = np.where(test.Sex=='male',True,False)
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Age_mean,Embarked_S,Embarked_C,Embarked_Q,FamilySize,Family,Family_S,Family_M,Family_L,Gender
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,22.0,True,False,False,2,M,False,True,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,38.0,False,True,False,2,M,False,True,False,False


In [64]:
fn = ['Gender', 'Age_mean','Embarked_S', 'Embarked_C', 'Embarked_Q','Family_S', 'Family_M', 'Family_L']
xTrain = train[fn]
xTrain

,Gender,Age_mean,Embarked_S,Embarked_C,Embarked_Q,Family_S,Family_M,Family_L
0,True,22.000000,True,False,False,False,True,False
1,False,38.000000,False,True,False,False,True,False
2,False,26.000000,True,False,False,True,False,False
3,False,35.000000,True,False,False,False,True,False
4,True,35.000000,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...
886,True,27.000000,True,False,False,True,False,False
887,False,19.000000,True,False,False,True,False,False
888,False,29.699118,True,False,False,False,True,False
889,True,26.000000,False,True,False,True,False,False


In [65]:
yLabel = train['Survived']
xTest = test[fn]

In [66]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=3,random_state=2020)
model.fit(xTrain,yLabel)
prediction = model.predict(xTest)

In [55]:
prediction.shape

(418,)

In [67]:
test['Survived']=prediction
res = test[['PassengerId','Survived']]
res.to_csv('mysubmit.csv',index=False)

In [60]:
test.shape

(418, 22)

In [72]:
train.Fare.describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [88]:
new = pd.concat([train,pd.get_dummies(train.Pclass)],axis=1)
new.rename(columns={1:'P1',2:'P2',3:'P3'},inplace=True)
new.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked_Q,FamilySize,Family,Family_S,Family_M,Family_L,Gender,P1,P2,P3
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,False,2,M,False,True,False,True,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,False,2,M,False,True,False,False,1,0,0


In [78]:
new.columns

Index(['PassengerId',    'Survived',      'Pclass',        'Name',
               'Sex',         'Age',       'SibSp',       'Parch',
            'Ticket',        'Fare',       'Cabin',    'Embarked',
          'Age_mean',  'Embarked_S',  'Embarked_C',  'Embarked_Q',
        'FamilySize',      'Family',    'Family_S',    'Family_M',
          'Family_L',      'Gender',             1,             2,
                   3],
      dtype='object')

In [83]:
dir(pd.DataFrame().rename)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__wrapped__']